#### imports 

In [1]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
from IPython.display import clear_output,display

In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

import time

def request_bot(meeting_url,language=None):
    native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"
    meeting_info = client.request_bot(
        platform="google_meet",
        native_meeting_id=native_meeting_id,
        bot_name="Vexa",
        language=language,
        task = 'transcribe'
    )
    return meeting_info
def get_transcript(meeting_url):
    native_meeting_id = meeting_url.split("/")[-1]
    try:
        for _ in range(10):
            
                transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
                df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
                clear_output()
                display(df.sort_values('absolute_start_time').tail(10))
                time.sleep(1)

    except Exception as e:
        print(e)

def stop_bot(meeting_url):
    native_meeting_id = meeting_url.split("/")[-1]
    client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


#### agenda

#### single bot per user/meeting
#### concurrent_bots
1. create user with 2 concurrency
2. start 1 bot
3. request another bot for the same meeting immediately -> must fail
4. first bot -> must come, admit the bot
5. speak and test transcription
6. launch bot for another meeting, admit the bot, check trnascription
7. launch another bot for the third meeting -> must fail
8. stop bot meeting 2 and and launch bot 3, but come

#### stop bot works for requested and active meetings
1 start meeting 1, stop immediately, start meeting 2 ->bot must come


#### tests

In [3]:
meeting_urls = ["https://meet.google.com/xdy-txjo-fyh", 
            "https://meet.google.com/fjp-cbnn-hug",
            "https://meet.google.com/wjf-bhmw-bix"]

##### 1. create user with 2 concurrency

In [4]:

new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", 
                                    name="test",
                                    max_concurrent_bots=2)
user_id = new_user['id']
user_id
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']
client = VexaClient(
    base_url=url,
    api_key=user_api_key)


DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '484002@example.com', 'name': 'test', 'max_concurrent_bots': 2}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 13:58:56 GMT, Wed, 03 Sep 2025 13:58:56 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '146', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"484002@example.com","name":"test","image_url":null,"max_concurrent_bots":2,"data":{},"id":123,"created_at":"2025-09-03T13:58:57.147251"}...

DEBUG: Making POST request to http://localhost:18056/admin/users/123/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 13:58:56 GMT, Wed, 03 Sep 2025 13:58:56 GMT', 'server': 'uvicorn, uv

In [5]:
#2. start 1 bot
request_bot(meeting_urls[0])
#3. request another bot for the same meeting immediately (must fail)
try:request_bot(meeting_urls[0])
except Exception as e:
    print(e) #An active or requested meeting already exists for this platform and meeting ID


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'xdy-txjo-fyh', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 13:59:03 GMT, Wed, 03 Sep 2025 13:59:03 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":246,"user_id":123,"platform":"google_meet","native_meeting_id":"xdy-txjo-fyh","constructed_meeting_url":"https://meet.google.com/xdy-txjo-fyh","status":"requested","bot_container_id":"4b77ecd17ce73fbfc1cb5ed1ae50b499933d675ccb261c739a1b8c85a5b2b55c","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-03T13:59:03.741186","updated_at":"2025-09-03T13:59:03.750442"}...

DEBUG: Making POST request to http://localhost:18056/bo

4. first bot must come, admit the bot and speak

In [6]:
time.sleep(15)
#5. speak and test transcription
get_transcript(meeting_urls[0])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
6,12.23,14.23,"Cześć, Dzień dobry.",pl,None,Dmitriy Grankin,2025-09-03T13:59:50.553106+00:00,2025-09-03T13:59:52.553106+00:00
7,12.53,13.00,"Czau, gliano.",pl,None,Dmitriy Grankin,2025-09-03T13:59:50.853106+00:00,2025-09-03T13:59:51.325106+00:00
8,14.23,16.23,"Nie, nie, nie.",pl,None,Dmitriy Grankin,2025-09-03T13:59:52.553106+00:00,2025-09-03T13:59:54.553106+00:00
9,16.23,18.23,To wszystko.,pl,None,Dmitriy Grankin,2025-09-03T13:59:54.553106+00:00,2025-09-03T13:59:56.553106+00:00
10,18.23,21.23,"A tym bardziej, co? Przedałem?",pl,None,Dmitriy Grankin,2025-09-03T13:59:56.553106+00:00,2025-09-03T13:59:59.553106+00:00
11,21.23,22.23,Dwa czasy więcej?,pl,None,None,2025-09-03T13:59:59.553106+00:00,2025-09-03T14:00:00.553106+00:00
12,22.23,25.23,Przedajesz?,pl,None,Dmitriy Grankin,2025-09-03T14:00:00.553106+00:00,2025-09-03T14:00:03.553106+00:00
13,23.23,24.23,Przedałem.,pl,None,Dmitriy Grankin,2025-09-03T14:00:01.553106+00:00,2025-09-03T14:00:02.553106+00:00
14,25.23,27.57,Dzień dobry.,pl,None,Dmitriy Grankin,2025-09-03T14:00:03.553106+00:00,2025-09-03T14:00:05.893106+00:00
15,25.57,27.57,Dzień dobry.,pl,None,Dmitriy Grankin,2025-09-03T14:00:03.893106+00:00,2025-09-03T14:00:05.893106+00:00


In [7]:
#6. launch bot for another meeting, admit the bot, check trnascription
request_bot(meeting_urls[1])



DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'fjp-cbnn-hug', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:00:11 GMT, Wed, 03 Sep 2025 14:00:10 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '384', 'content-type': 'application/json'}
DEBUG: Response content: {"id":247,"user_id":123,"platform":"google_meet","native_meeting_id":"fjp-cbnn-hug","constructed_meeting_url":"https://meet.google.com/fjp-cbnn-hug","status":"requested","bot_container_id":"0645f42e2fc7b14ece9fadeabd589b5cb73dce6f8b5b580445b7089de8cf47f8","start_time":null,"end_time":null,"data":{},"created_at":"2025-09-03T14:00:11.706741","updated_at":"2025-09-03T14:00:11.715316"}...


{'id': 247,
 'user_id': 123,
 'platform': 'google_meet',
 'native_meeting_id': 'fjp-cbnn-hug',
 'constructed_meeting_url': 'https://meet.google.com/fjp-cbnn-hug',
 'status': 'requested',
 'bot_container_id': '0645f42e2fc7b14ece9fadeabd589b5cb73dce6f8b5b580445b7089de8cf47f8',
 'start_time': None,
 'end_time': None,
 'data': {},
 'created_at': '2025-09-03T14:00:11.706741',
 'updated_at': '2025-09-03T14:00:11.715316'}

In [10]:
time.sleep(15)
get_transcript(meeting_urls[1])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,2.14,6.14,"Грин, ты всё, блин, в Черноховом озере всё равно синий шерел.",ru,None,Dmitriy Grankin,2025-09-03T14:01:16.695306+00:00,2025-09-03T14:01:20.695306+00:00
1,6.14,7.52,А вы всё...,ru,None,Dmitriy Grankin,2025-09-03T14:01:20.695306+00:00,2025-09-03T14:01:22.082306+00:00
2,6.64,8.64,А вы всё упражняетесь?,ru,None,Dmitriy Grankin,2025-09-03T14:01:21.195306+00:00,2025-09-03T14:01:23.195306+00:00
3,8.64,14.64,"Ну всё, думаю, надеемся, что когда-нибудь ими займутся.",ru,None,Dmitriy Grankin,2025-09-03T14:01:23.195306+00:00,2025-09-03T14:01:29.195306+00:00
4,14.64,21.96,"Все надеетесь, что им когда-нибудь займутся.",ru,None,Dmitriy Grankin,2025-09-03T14:01:29.195306+00:00,2025-09-03T14:01:36.515306+00:00
5,16.96,21.96,"Все надеетесь, что им когда-нибудь займутся.",ru,None,Dmitriy Grankin,2025-09-03T14:01:31.515306+00:00,2025-09-03T14:01:36.515306+00:00


In [11]:
#7. launch another bot for the third meeting -> must fail
try:request_bot(meeting_urls[2])
except Exception as e:
    print(e) #User has reached the maximum concurrent bot limit (2).


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'wjf-bhmw-bix', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 403
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:01:45 GMT, Wed, 03 Sep 2025 14:01:46 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"detail":"User has reached the maximum concurrent bot limit (2)."}...
HTTP Error 403 for POST http://localhost:18056/bots: User has reached the maximum concurrent bot limit (2).


In [12]:
#8. stop bot meeting 2 and and launch bot 3, but come
stop_bot(meeting_urls[1])
request_bot(meeting_urls[2])


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/fjp-cbnn-hug
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:01:51 GMT, Wed, 03 Sep 2025 14:01:51 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...

DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'wjf-bhmw-bix', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:01:51 GMT, Wed, 03 Sep 2025 14:01:51 GMT', 'server': 'uvicor

{'id': 248,
 'user_id': 123,
 'platform': 'google_meet',
 'native_meeting_id': 'wjf-bhmw-bix',
 'constructed_meeting_url': 'https://meet.google.com/wjf-bhmw-bix',
 'status': 'requested',
 'bot_container_id': 'bb4e21fac7cc692677b687c93617f18b7ea26889e6567b578a7e54d1431cacb9',
 'start_time': None,
 'end_time': None,
 'data': {},
 'created_at': '2025-09-03T14:01:51.740542',
 'updated_at': '2025-09-03T14:01:51.747629'}

In [14]:
time.sleep(15)
get_transcript(meeting_urls[2])

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,2.28,"three, four, five six, seven eight nine ten.",en,None,Dmitriy Grankin,2025-09-03T14:02:16.378812+00:00,2025-09-03T14:02:18.658812+00:00
1,2.28,4.22,"One, two, three, four, five six,",en,None,Dmitriy Grankin,2025-09-03T14:02:18.658812+00:00,2025-09-03T14:02:20.598812+00:00
2,4.22,5.84,seven eight nine ten.,en,None,Dmitriy Grankin,2025-09-03T14:02:20.598812+00:00,2025-09-03T14:02:22.218812+00:00
3,5.84,10.60,1 2 3 4 5 6 7 8 9 10,en,None,Dmitriy Grankin,2025-09-03T14:02:22.218812+00:00,2025-09-03T14:02:26.978812+00:00
4,10.60,14.60,1 2 3 4 5 6 7 8 9 10,en,None,Dmitriy Grankin,2025-09-03T14:02:26.978812+00:00,2025-09-03T14:02:30.978812+00:00
5,14.60,15.60,10 8,en,None,Dmitriy Grankin,2025-09-03T14:02:30.978812+00:00,2025-09-03T14:02:31.978812+00:00
6,14.68,18.32,10-8-7-8,en,None,Dmitriy Grankin,2025-09-03T14:02:31.058812+00:00,2025-09-03T14:02:34.698812+00:00


In [15]:
running_bots = client.get_running_bots_status()
len(running_bots) #should be 2



DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:02:41 GMT, Wed, 03 Sep 2025 14:02:42 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '867', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"bb4e21fac7cc692677b687c93617f18b7ea26889e6567b578a7e54d1431cacb9","container_name":"vexa-bot-248-d29af80a","platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","status":"Up 50 seconds","normalized_status":"Up","created_at":"2025-09-03T14:01:51+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"123"},"meeting_id_from_name":"248"},{"container_id":"4b77ecd17ce73fbfc1cb5ed1ae50b499933d675cc...


2

In [16]:
running_bots

[{'container_id': 'bb4e21fac7cc692677b687c93617f18b7ea26889e6567b578a7e54d1431cacb9',
  'container_name': 'vexa-bot-248-d29af80a',
  'platform': 'google_meet',
  'native_meeting_id': 'wjf-bhmw-bix',
  'status': 'Up 50 seconds',
  'normalized_status': 'Up',
  'created_at': '2025-09-03T14:01:51+00:00',
  'labels': {'org.opencontainers.image.ref.name': 'ubuntu',
   'org.opencontainers.image.version': '22.04',
   'vexa.user_id': '123'},
  'meeting_id_from_name': '248'},
 {'container_id': '4b77ecd17ce73fbfc1cb5ed1ae50b499933d675ccb261c739a1b8c85a5b2b55c',
  'container_name': 'vexa-bot-246-7c72d729',
  'platform': 'google_meet',
  'native_meeting_id': 'xdy-txjo-fyh',
  'status': 'Up 3 minutes',
  'normalized_status': 'Up',
  'created_at': '2025-09-03T13:59:03+00:00',
  'labels': {'org.opencontainers.image.ref.name': 'ubuntu',
   'org.opencontainers.image.version': '22.04',
   'vexa.user_id': '123'},
  'meeting_id_from_name': '246'}]

In [18]:
meetings = client.get_meetings()
[m['status'] for m in meetings] == ['active', 'completed', 'active']


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:02:44 GMT, Wed, 03 Sep 2025 14:02:45 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1183', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":248,"user_id":123,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"active","bot_container_id":"vexa-bot-248-d29af80a","start_time":"2025-09-03T14:02:15.221559","end_time":null,"data":{},"created_at":"2025-09-03T14:01:51.740542","updated_at":"2025-09-03T14:02:15.219386"},{"id":247,"user_id":123,"platform":"google_meet","native_meeting_id":"fjp-cbnn-hug","constructed_meeting_url":"https://meet....


True

In [19]:
for m in meeting_urls:
    try:stop_bot(m)
    except Exception as e:
        print(e)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/xdy-txjo-fyh
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:02:47 GMT, Wed, 03 Sep 2025 14:02:47 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...

DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/fjp-cbnn-hug
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 404
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:02:47 GMT, Wed, 03 Sep 2025 14:02:47 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '38', 'content-type': 'application/json'}
DE

In [20]:
meetings = client.get_meetings()
[m['status'] for m in meetings] == ['completed', 'completed', 'completed']


DEBUG: Making GET request to http://localhost:18056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'lg4MiLTaQEIdpfzgSzyb9mpObCAtLwqB1vEm8O2O'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 03 Sep 2025 14:02:50 GMT, Wed, 03 Sep 2025 14:02:50 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1343', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":248,"user_id":123,"platform":"google_meet","native_meeting_id":"wjf-bhmw-bix","constructed_meeting_url":"https://meet.google.com/wjf-bhmw-bix","status":"completed","bot_container_id":"vexa-bot-248-d29af80a","start_time":"2025-09-03T14:02:15.221559","end_time":"2025-09-03T14:02:49.424428","data":{"languages":["en"],"participants":["Dmitriy Grankin"]},"created_at":"2025-09-03T14:01:51.740542","updated_at":"2025-09-03T14:02:49.430038"},{"id":247,"user_id":123,"platform":"google_m...


True